In [1]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import numpy as np
import pandas as pd
import time
import re
from tqdm import tqdm
import json 

# Query "Robert Enke"

In [205]:
json_list=[]
seed_page="https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page="
attributes=["title","thema1","thema2","date","text","link"]
json_file={k:np.nan for k in attributes}

for page_no in range(1,10,1):
    
    print("Page number: {}".format(page_no))
    print(seed_page+str(page_no))
    page=requests.get(seed_page+str(page_no))

    page_content=BeautifulSoup(page.content,"html.parser")

    for row in page_content.find_all("div",{"class":"pdb-teaser pdb-teaser3-row row"}):
        
        for pages in row.find_all("div",{"class":"pdb-teaser3-row-item col-4"}):
            for element in pages.find_all("div",class_="pdb-teaser3-teaser"):
                json_file={k:np.nan for k in attributes}
                #thema 1
                try:
                    json_file["thema1"]=element.find_all("a",{"class":"pdb-teaser3-teaser-breadcrumb-frontpage"})[0].text
                except:
                    json_file["thema1"]=element.find_all("span",class_="pdb-parts-paidcontent-freeuntilbadge pdb-parts-paidcontent-freeuntilbadge_teaser3 pdb-parts-paidcontent-freeuntilbadge_close")[0].text
                #Thema2
                try:
                    json_file["thema2"]=element.find_all("span",class_="pdb-teaser3-teaser-breadcrumb-headline-shorttitle")[0].text
                except:
                    json_file["thema2"]="KN_PLUS"
                #title
                try:
                    json_file["title"]=element.find_all("h3",{"class":"pdb-teaser3-teaser-breadcrumb-headline"})[0].find_all("a")[0].text
                except:
                    json_file["title"]=element.find_all("div",class_="pdb-richtext-field")[0].text
                #date
                json_file["date"]=element.find_all("span",{"class":"pdb-teaser3-teaser-release-publicationdate"})[0].text
                #text
                json_file["text"]=element.find_all("div",{"class":"pdb-richtext-field"})[0].text
                #link
                if element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")[:1]=="/":
                    json_file["link"]="https://www.kn-online.de"+str(element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href"))
                else:
                    json_file["link"]=element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")
                json_list.append(json_file)

Page number: 1
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=1
Page number: 2
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=2
Page number: 3
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=3
Page number: 4
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=4
Page number: 5
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=5
Page number: 6
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=6
Page number: 7
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=7
Page number: 8
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=8
Page number: 9
https://www.kn-online.de/suche?query=robert+enke&in=all&sort=article&date=-1&page=9


In [206]:
df=pd.DataFrame(json_list)

In [207]:
df.apply(lambda x: x["date"].rstrip().replace(" ","")[0:],axis=1)

0     \n25.04.2020
1     \n27.03.2020
2     \n01.02.2020
3     \n06.12.2019
4     \n10.11.2019
          ...     
61    \n03.05.2012
62    \n15.02.2012
63    \n03.12.2009
64    \n13.11.2009
65    \n04.02.2009
Length: 66, dtype: object

In [208]:
df["date"]=df.apply(lambda x: pd.to_datetime(x["date"].rstrip().replace(" ","")[1:],format="%d.%m.%Y"),axis=1)

In [209]:
df

,title,thema1,thema2,date,text,link
0,Gladbach-Star Matthias Ginter über die Corona-...,Sport,SPORTBUZZER-Interview,2020-04-25,Nationalspieler Matthias Ginter (26) von Borus...,https://www.sportbuzzer.de/artikel/borussia-mo...
1,Schaaf über Lippenbekenntnisse in Corona-Krise...,Sport,Werder-Legende,2020-03-27,Thomas Schaaf (58) trainierte knapp 14 Jahre l...,https://www.sportbuzzer.de/artikel/werder-brem...
2,Illegal entsorgter Müll: Mehr als 15 000 Anzeigen,Hamburg,Mülldetektive in Hamburg,2020-02-01,Die Hamburger Stadtreinigung hat 2019 mehr als...,https://www.kn-online.de/Nachrichten/Hamburg/I...
3,1-Live-Krone-Preisverleihung: Jugend hört am l...,Medien & TV,Musik,2019-12-06,Zum 20. Mal haben Radiohörer Musikern der deut...,https://www.rnd.de/medien/1live-krone-preisver...
4,Gedenken: Löw und Witwe Teresa besuchen Grab v...,Sport,Kranz-Niederlegung,2019-11-10,Bundestrainer Joachim Löw hat zehn Jahre nach ...,https://www.sportbuzzer.de/artikel/robert-enke...
...,...,...,...,...,...,...
61,Die Bundesliga-Saison ist fast zu Ende. Sie hi...,\n Fußball\n ...,KN_PLUS,2012-05-03,Die Bundesliga-Saison ist fast zu Ende. Sie hi...,https://www.kn-online.de/Sport/Sportmix/Reus-R...
62,Andreas Biermanns Bekenntnis via Facebook löst...,\n Fußball\n ...,KN_PLUS,2012-02-15,Andreas Biermanns Bekenntnis via Facebook löst...,https://www.kn-online.de/Sport/Sportmix/Betrof...
63,Wenn das Telefon in der neuen THW-Geschäftsste...,\n Michael Menzel m...,KN_PLUS,2009-12-03,Wenn das Telefon in der neuen THW-Geschäftsste...,https://www.kn-online.de/Sport/THW-Kiel/Nach-m...
64,Hamburg - Der Mikrokosmos Profi-Fußball will s...,\n Nach dem Suizid ...,KN_PLUS,2009-11-13,Hamburg - Der Mikrokosmos Profi-Fußball will s...,https://www.kn-online.de/Sport/Sportmix/Fussba...


In [210]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\kn_query_[Robert_Enke].xlsx') as writer:
    df.to_excel(writer)

# Query Depression

In [220]:
json_list=[]
seed_page="https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page="
attributes=["title","thema1","thema2","date","text","link"]
json_file={k:np.nan for k in attributes}

for page_no in range(1,11,1):
    
    print("Page number: {}".format(page_no))
    print(seed_page+str(page_no))
    page=requests.get(seed_page+str(page_no))

    page_content=BeautifulSoup(page.content,"html.parser")

    for row in page_content.find_all("div",{"class":"pdb-teaser pdb-teaser3-row row"}):
        
        for pages in row.find_all("div",{"class":"pdb-teaser3-row-item col-4"}):
            for element in pages.find_all("div",class_="pdb-teaser3-teaser"):
                json_file={k:np.nan for k in attributes}
                #thema 1
                try:
                    json_file["thema1"]=element.find_all("a",{"class":"pdb-teaser3-teaser-breadcrumb-frontpage"})[0].text
                except:
                    json_file["thema1"]=element.find_all("span",class_="pdb-parts-paidcontent-freeuntilbadge pdb-parts-paidcontent-freeuntilbadge_teaser3 pdb-parts-paidcontent-freeuntilbadge_close")[0].text
                #Thema2
                try:
                    json_file["thema2"]=element.find_all("span",class_="pdb-teaser3-teaser-breadcrumb-headline-shorttitle")[0].text
                except:
                    json_file["thema2"]="KN_PLUS"
                #title
                try:
                    json_file["title"]=element.find_all("h3",{"class":"pdb-teaser3-teaser-breadcrumb-headline"})[0].find_all("a")[0].text
                except:
                    json_file["title"]=element.find_all("div",class_="pdb-richtext-field")[0].text
                #date
                json_file["date"]=element.find_all("span",{"class":"pdb-teaser3-teaser-release-publicationdate"})[0].text
                #text
                json_file["text"]=element.find_all("div",{"class":"pdb-richtext-field"})[0].text
                #link
                if element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")[:1]=="/":
                    json_file["link"]="https://www.kn-online.de"+str(element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href"))
                else:
                    json_file["link"]=element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")
                json_list.append(json_file)

Page number: 1
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=1
Page number: 2
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=2
Page number: 3
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=3
Page number: 4
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=4
Page number: 5
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=5
Page number: 6
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=6
Page number: 7
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=7
Page number: 8
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=8
Page number: 9
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=9
Page number: 10
https://www.kn-online.de/suche?query=Depression&in=all&sort=article&date=-1&page=10


In [221]:
df=pd.DataFrame(json_list)

In [222]:
df

,title,thema1,thema2,date,text,link
0,Krisen besser meistern: So befreit man sich vo...,Gesundheit,Wissenschaft,\n ...,Die eigene Stimmung in der Balance zu halten i...,https://www.rnd.de/gesundheit/depressionen-und...
1,Hat uns die Corona-Krise zu anderen Menschen g...,Gesundheit,Coronavirus,\n ...,Am Anfang war die Hoffnung groß: Die Corona-Kr...,https://www.rnd.de/gesundheit/hat-uns-die-coro...
2,"Stefanie Kreutzer war vier Jahre alt, als ihr ...",\n Interview mit Be...,KN_PLUS,\n ...,"Stefanie Kreutzer war vier Jahre alt, als ihr ...",https://www.kn-online.de/Nachrichten/Schleswig...
3,Keine Veränderung beim Leitzins: US-Notenbank ...,Wirtschaft,USA,\n ...,Experten hatten es bereits erwartet. Die US-No...,https://www.rnd.de/wirtschaft/leitzins-keine-v...
4,"Hinhören, ernst nehmen: Opfer von Kindesmissbr...",Panorama,Missbrauch,\n ...,Werden Fälle von Kindesmissbrauch wie der in M...,https://www.rnd.de/panorama/hinhoren-ernst-neh...
...,...,...,...,...,...,...
75,Einlagensicherung: So wird das Geld auf dem Ko...,Wirtschaft,Coronavirus,\n ...,Wegen des Coronavirus leidet die Wirtschaft – ...,https://www.rnd.de/wirtschaft/einlagensicherun...
76,Finanzprofi Tenhagen: “Ich wäre jetzt auch vor...,Wirtschaft,Coronavirus,\n ...,"Die Konten sind sicher, viele Arbeitsplätze ni...",https://www.rnd.de/wirtschaft/finanzprofi-tenh...
77,VW: Tarifabschluss ohne Lohnerhöhung,Wirtschaft,Volkswagen,\n ...,Die knapp 120.000 Beschäftigten von Volkswagen...,https://www.rnd.de/wirtschaft/vw-tarifabschlus...
78,IWF-Chefin: Schlimmster Wirtschaftscrash seit ...,Wirtschaft,IWF,\n ...,Der Internationale Währungsfonds (IWF) rechnet...,https://www.rnd.de/wirtschaft/kommt-die-wirtsc...


In [223]:
def transform_date(x):
    try:
        x=pd.to_datetime(x.rstrip().replace(" ","")[1:],format="%d.%m.%Y")
    except:
        pass
    return x

In [224]:
df["date"]=df.apply(lambda x: transform_date(x["date"]),axis=1)

In [225]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\kn_query_[Depression].xlsx') as writer:
    df.to_excel(writer)

# Query Suizid

In [229]:
json_list=[]
seed_page="https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page="
attributes=["title","thema1","thema2","date","text","link"]
json_file={k:np.nan for k in attributes}

for page_no in range(1,11,1):
    
    print("Page number: {}".format(page_no))
    print(seed_page+str(page_no))
    page=requests.get(seed_page+str(page_no))

    page_content=BeautifulSoup(page.content,"html.parser")

    for row in page_content.find_all("div",{"class":"pdb-teaser pdb-teaser3-row row"}):
        
        for pages in row.find_all("div",{"class":"pdb-teaser3-row-item col-4"}):
            for element in pages.find_all("div",class_="pdb-teaser3-teaser"):
                json_file={k:np.nan for k in attributes}
                #thema 1
                try:
                    json_file["thema1"]=element.find_all("a",{"class":"pdb-teaser3-teaser-breadcrumb-frontpage"})[0].text
                except:
                    json_file["thema1"]=element.find_all("span",class_="pdb-parts-paidcontent-freeuntilbadge pdb-parts-paidcontent-freeuntilbadge_teaser3 pdb-parts-paidcontent-freeuntilbadge_close")[0].text
                #Thema2
                try:
                    json_file["thema2"]=element.find_all("span",class_="pdb-teaser3-teaser-breadcrumb-headline-shorttitle")[0].text
                except:
                    json_file["thema2"]="KN_PLUS"
                #title
                try:
                    json_file["title"]=element.find_all("h3",{"class":"pdb-teaser3-teaser-breadcrumb-headline"})[0].find_all("a")[0].text
                except:
                    json_file["title"]=element.find_all("div",class_="pdb-richtext-field")[0].text
                #date
                json_file["date"]=element.find_all("span",{"class":"pdb-teaser3-teaser-release-publicationdate"})[0].text
                #text
                json_file["text"]=element.find_all("div",{"class":"pdb-richtext-field"})[0].text
                #link
                if element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")[:1]=="/":
                    json_file["link"]="https://www.kn-online.de"+str(element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href"))
                else:
                    json_file["link"]=element.find_all("span",{"class":"pdb-teaser3-teaser-breadcrumb-headline-title"})[0].find_all("a")[0].get("href")
                json_list.append(json_file)

Page number: 1
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=1
Page number: 2
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=2
Page number: 3
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=3
Page number: 4
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=4
Page number: 5
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=5
Page number: 6
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=6
Page number: 7
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=7
Page number: 8
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=8
Page number: 9
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=9
Page number: 10
https://www.kn-online.de/suche?query=Suizid&in=all&sort=article&date=-1&page=10


In [230]:
df=pd.DataFrame(json_list)

In [231]:
df

,title,thema1,thema2,date,text,link
0,Eltern finden in Bayern zwei tote Jugendliche,Panorama,Polizei,\n ...,Eltern haben in der Nähe von Augsburg ihren 16...,https://www.rnd.de/panorama/eltern-finden-in-b...
1,Ein Junge (10) ist am Sonntag in Stockelsdorf ...,\n Stockelsdorf\n ...,KN_PLUS,\n ...,Ein Junge (10) ist am Sonntag in Stockelsdorf ...,https://www.kn-online.de/Lokales/Ostholstein/S...
2,Billie Eilish hat sich noch nie körperlich beg...,Promis,Billie Eilish,\n ...,Billie Eilish setzt sich vehement gegen Bodysh...,https://www.rnd.de/promis/billie-eilish-hat-si...
3,Nach Schüssen in Kärnten: Motiv des Schützen n...,Panorama,Österreich,\n ...,Innerhalb weniger Stunden tötet ein 63-Jährige...,https://www.rnd.de/panorama/osterreich-nach-sc...
4,So dramatisch endet die Netflix-Serie “Tote Mä...,Medien & TV,Netflix,\n ...,Mit Staffel vier endet die düstere Teenagerser...,https://www.rnd.de/medien/so-dramatisch-endet-...
...,...,...,...,...,...,...
75,Sterbehilfe: Spahn will verfassungsgemäße Lösu...,Politik,Bundesverfassungsgericht,\n ...,Das Bundesverfassungsgericht hat das Verbot or...,https://www.rnd.de/politik/sterbehilfe-spahn-w...
76,Mitkläger Brennecke zum Sterbehilfeurteil: “Üb...,Politik,Sterbehilfe,\n ...,Mit der Legalisierung organisierter Sterbehilf...,https://www.rnd.de/politik/mitklager-brennecke...
77,Moderator Jürgen Domian begrüßt Urteil zu Ster...,Promis,Sterbehilfe,\n ...,Seit Jahren setzt sich Jürgen Domian für das R...,https://www.rnd.de/promis/moderator-jurgen-dom...
78,Sterbehilfe-Urteil: Ärzte wollen Beihilfe-Verb...,Politik,Sterbehilfe,\n ...,Das Verbot organisierter Hilfe beim Suizid ist...,https://www.rnd.de/politik/nach-karlsruher-urt...


In [232]:
df=pd.DataFrame(json_list)
df["date"]=df.apply(lambda x: transform_date(x["date"]),axis=1)
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\kn_query_[Suizid].xlsx') as writer:
    df.to_excel(writer)

# Liste aller KN-Artikel des Zeitraums 01.01.2009 - 31.12.2009 auf Basis des KN-Archivs erstellen

In [5]:
json_list=[]
seed_page="https://www.kn-online.de/Mehr/Archiv/"
attributes=["title_1","title_2","main_category","date","link","paywall_status"]
json_file={k:np.nan for k in attributes}

for date in pd.date_range(start="2019-01-01",end="2019-12-31"):
    #construct date
    if date.month<10:
        if date.day<10:
            datum=str(date.year)+"/0"+str(date.month)+"/0"+str(date.day)
        else:
            datum=str(date.year)+"/0"+str(date.month)+"/"+str(date.day)
    else:
        if date.day<10:
            datum=str(date.year)+"/"+str(date.month)+"/0"+str(date.day)
        else:
            datum=str(date.year)+"/"+str(date.month)+"/"+str(date.day)
    #create page with date
    new_page=seed_page+datum
    print(new_page)
    page=requests.get(new_page)
    page_content=BeautifulSoup(page.content,"html.parser")

    for news_element in page_content.find_all("div",class_="pdb-full-newsticker-teaser"):
        json_file={k:np.nan for k in attributes}
        #print(news_element)
        try:
            json_file["date"]=news_element.find_all("span",class_="pdb-full-newsticker-teaser-release-publicationdate")[0].text
            #print(json_file["date"])
        except:
            json_file["date"]=0
            print("extraction failed for date")
        try:
            json_file["title_1"]=news_element.find_all("span",class_="pdb-parts-paidcontent-freeuntilbadge pdb-parts-paidcontent-freeuntilbadge_full-newsticker pdb-parts-paidcontent-freeuntilbadge_close")[0].text.rstrip()[29:]
        except:
            json_file["title_1"]=0
            print("extraction failed for title_1")        
        try:
            json_file["title_2"]=news_element.find_all("span","pdb-full-newsticker-teaser-breadcrumb-headline-title")[0].text
        except:
            json_file["title_2"]=0
            print("extraction failed for title_2") 
        try:
            json_file["link"]="https://www.kn-online.de"+str(news_element.find_all("a")[0].get("href"))
        except:
            json_file["link"]=0
            print("extraction failed for link") 
        try:
            json_file["main_category"]=json.loads(news_element.find_all("a")[0].get("data-event"))["articleMainCategory"]
        except:
            json_file["main_category"]=0
            print("extraction failed for main cat") 
        try:
            json_file["paywall_status"]=json.loads(news_element.find_all("a")[0].get("data-event"))["articlePaywallStatus"]
        except:
            json_file["paywall_status"]
            print("extraction failed for paywall_status") 
        json_list.append(json_file)

 failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat
extraction failed for paywall_status
extraction failed for title_1
extraction failed for main cat

ConnectionError: ('Connection aborted.', OSError("(10051, 'WSAENETUNREACH')"))

In [ ]:
df=pd.DataFrame(json_list)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
with pd.ExcelWriter(r'C:\Users\rbwes\Desktop\Yannick_Daten\KN\kn_archiv_query_2019.xlsx') as writer:
    df.to_excel(writer)